In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import scipy.stats as scs
from sklearn.preprocessing import Imputer

In [2]:
df = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/properties_2016.csv')
target = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/train_2016_v2.csv')
joined_data = pd.merge(target, df, on="parcelid", how="left")

In [3]:
df2 = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/properties_2017.csv')
target2 = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/train_2017_v2.csv')
joined_data2 = pd.merge(target2, df2, on="parcelid", how="left")

In [ ]:
# are there properties solde more than once?  Should we create new observations?
target['parcelid'].value_counts().reset_index()['parcelid'].value_counts()

In [96]:
# joining data
joined_data = pd.merge(target,df,on="parcelid",how="left")

In [5]:
# saving memory
for c, dtype in zip(joined_data.columns, joined_data.dtypes):
    if dtype == np.float64:
        joined_data[c] = joined_data[c].astype(np.float32)

In [6]:
#--- Creating two additional columns each for the month and day ---
joined_data.transactiondate = pd.to_datetime(joined_data.transactiondate,format='%Y-%m-%d')

joined_data['transaction_year'] = joined_data.transactiondate.dt.weekday.astype(np.int64)
joined_data['transaction_month'] = joined_data.transactiondate.dt.month.astype(np.int64)
joined_data['transaction_dow'] = joined_data.transactiondate.dt.weekday.astype(np.int64)

#--- Dropping the 'transactiondate' column now ---
joined_data = joined_data.drop('transactiondate', 1)

In [267]:
per_null = joined_data.isnull().sum()/len(joined_data)
per_null.sort_values(ascending=True)

parcelid                        0.000000
block                           0.000000
censustract                     0.000000
censustractandblock             0.000000
assessmentyear                  0.000000
roomcnt                         0.000000
regionidcounty                  0.000000
rawcensustractandblock          0.000000
propertylandusetypeid           0.000000
censuscounty                    0.000000
longitude                       0.000000
latitude                        0.000000
fips                            0.000000
censusblock                     0.000000
logerror                        0.000000
transactiondate                 0.000000
bathroomcnt                     0.000000
bedroomcnt                      0.000000
landtaxvaluedollarcnt           0.000011
taxvaluedollarcnt               0.000011
propertycountylandusecode       0.000011
taxamount                       0.000066
regionidzip                     0.000388
structuretaxvaluedollarcnt      0.004209
calculatedfinish

In [7]:
num_cols = ['logerror', 'roomcnt', 'bathroomcnt', 'fullbathcnt', 'bedroomcnt', 'calculatedbathnbr', 'threequarterbathnbr', 'fireplacecnt', 'garagecarcnt','numberofstories', 'poolcnt', 'unitcnt', 'finishedfloor1squarefeet', 'basementsqft', 'finishedsquarefeet12', 'calculatedfinishedsquarefeet', 'finishedsquarefeet13', 'finishedsquarefeet6', 'lotsizesquarefeet', 'yardbuildingsqft17', 'yardbuildingsqft26', 'poolsizesum', 'finishedsquarefeet15', 'finishedsquarefeet50', 'garagetotalsqft', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'taxamount', 'taxvaluedollarcnt']
num_data = joined_data[num_cols]
num_data.columns = ['logerror', 'room_cnt', 'bath_cnt', 'fullbath_cnt', 'bed_cnt', 'bathnbr_cnt','3/4bathnbr_cnt', 'fireplace_cnt', 'garagecar_cnt', 'stories_cnt', 'pool_cnt', 'unit_cnt', 'floor1_sqft', 'basement_sqft', 'totallvng1_sqft', 'totallvng2_sqft', 'perimlvng_sqft', 'basearea_sqft', 'lot_sqft', 'patio_sqft', 'yardbldg_sqft', 'pool_sqft', 'total_sqft', 'finishedsqft50', 'garagetotal_sqft', 'landtaxvalue_$', 'strcrtaxvalue_$', 'tax_$', 'taxvalue_$']

In [ ]:
for col in num_data.columns:
    print(col, ':', num_data[col].nunique())

In [93]:
num_cols = ['logerror', 'roomcnt', 'bathroomcnt', 'fullbathcnt', 'bedroomcnt', 'calculatedbathnbr', 'threequarterbathnbr', 'fireplacecnt', 'garagecarcnt','numberofstories', 'unitcnt', 'finishedfloor1squarefeet', 'basementsqft', 'finishedsquarefeet12', 'calculatedfinishedsquarefeet', 'finishedsquarefeet13', 'finishedsquarefeet6', 'lotsizesquarefeet', 'yardbuildingsqft17', 'yardbuildingsqft26', 'poolsizesum', 'finishedsquarefeet15', 'finishedsquarefeet50', 'garagetotalsqft', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'taxamount', 'taxvaluedollarcnt']
num_data = joined_data[num_cols]
num_data.columns = ['logerror', 'room_cnt', 'bath_cnt', 'fullbath_cnt', 'bed_cnt', 'bathnbr_cnt','3/4bathnbr_cnt', 'fireplace_cnt', 'garagecar_cnt', 'stories_cnt', 'unit_cnt', 'floor1_sqft', 'basement_sqft', 'totallvng1_sqft', 'totallvng2_sqft', 'perimlvng_sqft', 'basearea_sqft', 'lot_sqft', 'patio_sqft', 'yardbldg_sqft', 'pool_sqft', 'total_sqft', 'finishedsqft50', 'garagetotal_sqft', 'landtaxvalue_$', 'strcrtaxvalue_$', 'tax_$', 'taxvalue_$']

In [8]:
## Bed, bath and room are the only variables with significant amout of outliers.  
outliers = {}
for col in num_data.columns:
    var = num_data[col].dropna()
    index = var[~((var-var.mean()).abs()<2*var.std())]
    outliers[col] = index

outliers = pd.DataFrame.from_dict(outliers).dropna(how='all')


def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    
    return np.where((ys > upper_bound) | (ys < lower_bound))

out_dict = {}
for col in num_data.columns:
    outlier = outliers_iqr(num_data[col])
    out_dict[col] = outlier
    
out_dict = dict((k, out_dict[k]) for k in ('bath_cnt','bed_cnt','room_cnt'))

bath = pd.DataFrame(list(list(out_dict['bath_cnt'])[0])).reset_index()
bed = pd.DataFrame(list(list(out_dict['bed_cnt'])[0])).reset_index()
room = pd.DataFrame(list(list(out_dict['room_cnt'])[0])).reset_index()

out_bbb = bath.merge(bed, on=0, how='outer').merge(room, on=0, how='outer')
out_bbb.columns = ['index', 'bath_cnt','bed_cnt','room_cnt']
out_bbb = out_bbb.iloc[:, 1:4]
# prob not anomalous: joined_data.loc[[14440, 42387, 60385]]

In [ ]:
## calculatedfinishedsquarefeet and finishedsquarefeet12 are identical
joined_data[joined_data.calculatedfinishedsquarefeet !=joined_data.finishedsquarefeet12][['calculatedfinishedsquarefeet', 'finishedsquarefeet12']].dropna()

In [ ]:
# finishedsquarefeet50 and finishedfloor1squarefeet are nearly identical and finishedsquarefeet50 has more nas
joined_data[joined_data.finishedsquarefeet50 !=joined_data.finishedfloor1squarefeet][['finishedsquarefeet50', 'finishedfloor1squarefeet']].dropna()

In [ ]:
## calculatedbathnbr same as bathroomcnt and has more nas
joined_data[joined_data.bathroomcnt !=joined_data.calculatedbathnbr][['bathroomcnt', 'calculatedbathnbr']].dropna()
joined_data[['bathroomcnt', 'calculatedbathnbr']].isnull().sum()

In [79]:
# threequarterbathnbr can be calculated from bathroomcnt - fullbathcnt
bth_compare = pd.DataFrame(joined_data['bathroomcnt'] - joined_data['fullbathcnt'])
bth_compare['34'] = joined_data['threequarterbathnbr']
bth_compare.columns = ['cnt', '34cnt']
bth_compare['cnt'] = bth_compare['cnt']*2
bth_compare.loc[pd.isnull(bth_compare['cnt']), 'cnt'] = 0
bth_compare.loc[pd.isnull(bth_compare['34cnt']), '34cnt'] = 0
bth_compare[bth_compare['cnt'] != bth_compare['34cnt']]

,cnt,34cnt


In [105]:
#basement is not included in the number of stories
joined_data.loc[pd.notnull(joined_data['basementsqft'])][['basementsqft', 'numberofstories']]

,basementsqft,numberofstories
460,814.0,3.0
3046,330.0,1.0
4033,168.0,1.0
4375,671.0,2.0
11120,1555.0,1.0
13361,1528.0,2.0
17639,771.0,1.0
17808,700.0,1.0
21398,831.0,1.0
21445,1048.0,1.0


In [43]:
#Fill in those properties that have a pool with median pool value
poolsizesum_median = joined_data.loc[joined_data['poolsizesum'] > 0, 'poolsizesum'].median()
joined_data.loc[(joined_data['poolsizesum'].isnull() | 
                joined_data['poolsizesum'] == 0) & 
            ((pd.notnull(joined_data['poolcnt'])) |
            (pd.notnull(joined_data['pooltypeid10'])) |
            (pd.notnull(joined_data['pooltypeid2'])) |
            (pd.notnull(joined_data['pooltypeid7']))), 'poolsizesum'] = poolsizesum_median

In [41]:
#Fill in those properties that have a pool size/type with pool count of 1
joined_data.loc[(joined_data['poolcnt'] != 1) & 
            ((pd.notnull(joined_data['poolsizesum'])) |
            (pd.notnull(joined_data['pooltypeid10'])) |
            (pd.notnull(joined_data['pooltypeid2'])) |
            (pd.notnull(joined_data['pooltypeid7']))), 'poolcnt'] = 1

In [10]:
# fill in fireplace flag when there is a fireplace count
joined_data[joined_data.fireplaceflag !=joined_data.fireplacecnt][['fireplaceflag', 'fireplacecnt']].dropna(how='all')
joined_data.loc[joined_data['fireplacecnt'] > 0].fireplaceflag = 'True'

In [11]:
# number of car garage is more that sqft
garage_median = joined_data[(joined_data.garagetotalsqft != 0)].garagetotalsqft.dropna().median()
joined_data.loc[(joined_data.garagetotalsqft == 0) & (joined_data.garagecarcnt > 0), 'garagetotalsqft'] = garage_median

In [12]:
# only 443 outliers for bedroom - replace with median
bed_ix = (list(list(out_dict['bed_cnt'])[0]))
bed_median = joined_data.iloc[~joined_data.index.isin(bed_ix)].bedroomcnt.median()
joined_data.loc[(bed_ix), 'bedroomcnt'] = bed_median

In [265]:
imp = Imputer(missing_values='NaN',  # Create imputation model
              strategy='mean',       # Use mean imputation
              axis=0)                # Impute by column
imp = imp.fit(num_data)
imputed_data = pd.DataFrame(imp.transform(num_data.values))

In [266]:
imputed_data

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0276,0.0,2.0,2.0,3.0,2.0,1.008743,1.187884,1.812005,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,483.178314,237416.0,122754.0,6735.879883,360170.0
1,-0.1684,0.0,3.5,3.0,4.0,3.5,1.000000,1.187884,2.000000,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,468.000000,239071.0,346458.0,10153.019531,585529.0
2,-0.0040,0.0,3.0,3.0,2.0,3.0,1.008743,1.187884,1.812005,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,483.178314,57912.0,61994.0,11484.480469,119906.0
3,0.0218,0.0,2.0,2.0,2.0,2.0,1.008743,1.187884,1.812005,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,483.178314,73362.0,171518.0,3048.739990,244880.0
4,-0.0050,8.0,2.5,2.0,4.0,2.5,1.000000,1.187884,2.000000,2.000000,...,310.14209,311.694733,500.000000,2380.090088,1355.546631,598.000000,264977.0,169574.0,5488.959961,434551.0
5,-0.2705,0.0,4.0,4.0,4.0,4.0,1.008743,1.187884,1.812005,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,483.178314,1567301.0,880650.0,27126.570312,2447951.0
6,0.0440,0.0,1.0,1.0,2.0,1.0,1.008743,1.187884,1.812005,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,483.178314,46972.0,64549.0,2304.969971,111521.0
7,0.1638,6.0,2.5,2.0,3.0,2.5,1.000000,1.000000,2.000000,2.000000,...,310.14209,311.694733,501.007935,2380.090088,853.000000,462.000000,199000.0,107000.0,3745.500000,306000.0
8,-0.0030,0.0,1.0,1.0,2.0,1.0,1.008743,1.187884,1.000000,1.440739,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,462.000000,143230.0,66834.0,2172.879883,210064.0
9,0.0843,5.0,2.0,2.0,2.0,2.0,1.008743,1.187884,1.000000,1.000000,...,310.14209,311.694733,501.007935,2380.090088,1355.546631,462.000000,80983.0,109977.0,1940.260010,190960.0


In [ ]:
impute_test = pd.DataFrame(impute_test,  # Remake DataFrame with new values
                           index=impute_test.index,
                           columns = impute_test.columns)

In [122]:
# unitcnt extreme outliers
unit_median = joined_data.loc[pd.notnull(joined_data['unitcnt'])]['unitcnt'].median()
joined_data.loc[(joined_data['unitcnt'] > 9), 'unitcnt'] = unit_median

In [127]:
# columns missing 99% of data
missing_df = joined_data.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / joined_data.shape[0]
missing_df.ix[missing_df['missing_ratio']>0.99]

,column_name,missing_count,missing_ratio
4,architecturalstyletypeid,90014,0.997109
5,basementsqft,90232,0.999524
8,buildingclasstypeid,90259,0.999823
11,decktypeid,89617,0.992711
15,finishedsquarefeet13,90242,0.999634
18,finishedsquarefeet6,89854,0.995336
43,storytypeid,90232,0.999524
45,typeconstructiontypeid,89976,0.996688
48,yardbuildingsqft26,90180,0.998948
51,fireplaceflag,90053,0.997541


In [144]:
# taxvaluedollarcnt is sum of structuretaxvaluedollarcnt and landtaxvaluedollarcnt
tax_assess = pd.DataFrame(joined_data['structuretaxvaluedollarcnt'] + joined_data['landtaxvaluedollarcnt'])
tax_assess['taxvaluedollarcnt'] = joined_data['taxvaluedollarcnt']
tax_assess

,0,taxvaluedollarcnt
0,360170.0,360170.0
1,585529.0,585529.0
2,119906.0,119906.0
3,244880.0,244880.0
4,434551.0,434551.0
5,2447951.0,2447951.0
6,111521.0,111521.0
7,306000.0,306000.0
8,210064.0,210064.0
9,190960.0,190960.0


In [151]:
joined_data[['regionidcity','regionidneighborhood','regionidzip', 'censustractandblock', 'latitude', 'longitude']]

,regionidcounty,regionidcity,regionidneighborhood,regionidzip,censustractandblock,latitude,longitude
0,3101.0,12447.0,31817.0,96370.0,6.037107e+13,34280990.0,-118488536.0
1,1286.0,32380.0,NaN,96962.0,NaN,33668120.0,-117677556.0
2,3101.0,47019.0,275411.0,96293.0,6.037464e+13,34136312.0,-118175032.0
3,3101.0,12447.0,54300.0,96222.0,6.037296e+13,33755800.0,-118309000.0
4,1286.0,17686.0,NaN,96961.0,6.059042e+13,33485643.0,-117700234.0
5,3101.0,29712.0,NaN,96109.0,6.037621e+13,33870089.0,-118402768.0
6,3101.0,24174.0,NaN,96091.0,6.037542e+13,33899475.0,-118212720.0
7,2061.0,13150.0,NaN,97101.0,6.111003e+13,34207204.0,-119165589.0
8,1286.0,25459.0,NaN,96987.0,6.059042e+13,33549600.0,-117678000.0
9,1286.0,46098.0,NaN,96963.0,6.059063e+13,33612700.0,-117742000.0


In [153]:
abs(joined_data[['regionidcounty','regionidcity','regionidneighborhood','regionidzip', 'censustractandblock', 'latitude', 'longitude']].dropna().corr())

,regionidcounty,regionidcity,regionidneighborhood,regionidzip,censustractandblock,latitude,longitude
regionidcounty,1.000000,0.210659,0.193711,0.862676,0.723088,0.573621,0.618259
regionidcity,0.210659,1.000000,0.281379,0.260657,0.148962,0.189444,0.286585
regionidneighborhood,0.193711,0.281379,1.000000,0.101307,0.177793,0.389008,0.305400
regionidzip,0.862676,0.260657,0.101307,1.000000,0.627336,0.308232,0.450520
censustractandblock,0.723088,0.148962,0.177793,0.627336,1.000000,0.261907,0.066403
latitude,0.573621,0.189444,0.389008,0.308232,0.261907,1.000000,0.705331
longitude,0.618259,0.286585,0.305400,0.450520,0.066403,0.705331,1.000000


In [258]:
joined_data['censustractandblock'] = joined_data['censustractandblock'].apply(str)
joined_data['censuscounty'] = joined_data['censustractandblock'].str[0:4]
joined_data.loc[joined_data['censustractandblock'].str[0:4]== 'nan', 'censuscounty'] = joined_data.loc[joined_data['censustractandblock'].str[0:4]== 'nan']['fips']
joined_data['censustract'] = joined_data['censustractandblock'].str[4:11]
joined_data['censusblock'] = joined_data['censustractandblock'].str[10:-2]